<table align="center">
  <td align="center"><a target="_blank" href="https://colab.research.google.com/drive/1yb4Wz9Jjsekh8NGAeqPd-cqsDl1w6ZZB#scrollTo=jAK0G5UrEZHl">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/SheilKumar/CFBPrediction">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>


# 0.0 Dependencies 

In [251]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#1.0 Objective 

The objective of this project will be to create a network to accurately predict the spread as well as the under and over for college footbal games. 



##1.0.1 Methodology 

* We will scrape the internet to find college football fixture results. 
* Clean the data
* Split data into training and testing sets. 
* Evaluate models. Expected output (Final Score, Spread) 

Expected Data Table : 

| Date | Home Team | Pts | Away Team  | Pts | Total Pts | Spread                       |
|------|-----------|-----|------------|-----|-----------|------------------------------|
|      | 1         | 4   | 5          |   20  |  Home Team Pts+Away Team Pts         |Away Team Pts- Home Team Pts |
|      | 2         | 8   | 20         |     23|           |                              |
|      | 3         | 12  | 13            |    43 |           |                              |


## 1.0.2 Begin Collection

* Change Winner Label to Home, Loser Label to away 
* Change Pts to Home_Pts, Pts.1 to Away_Pts 
* Switch Team and Pts Labels for those `@` 
* Create Total_Pts Variable 
* Create Spread Variable `Home_Pts - Away_Pts`


In [252]:
raw = pd.read_html("https://www.sports-reference.com/cfb/years/2020-schedule.html", header = 0)
df = raw[0]
completed_fixtures = df.drop(df[df.Pts=='Pts'].index)
completed_fixtures = completed_fixtures.rename(columns={'Pts':'Home_Pts', 'Pts.1':'Away_Pts','Winner':'Home','Loser':'Away'})
completed_fixtures = completed_fixtures.dropna(subset=['Home_Pts'])
completed_fixtures = completed_fixtures.drop(['Rk', 'Notes'], axis=1)
completed_fixtures = completed_fixtures.astype({'Home_Pts':'int32'})
completed_fixtures = completed_fixtures.astype({'Away_Pts':'int32'})
completed_fixtures = completed_fixtures.assign(Total_Pts = completed_fixtures['Home_Pts']+completed_fixtures['Away_Pts'])
completed_fixtures


,Wk,Date,Time,Day,Home,Home_Pts,Unnamed: 7,Away,Away_Pts,Total_Pts
0,1,"Sep 3, 2020",8:00 PM,Thu,Alabama-Birmingham,45,NaN,Central Arkansas,35,80
1,1,"Sep 3, 2020",8:00 PM,Thu,South Alabama,32,@,Southern Mississippi,21,53
2,1,"Sep 5, 2020",1:30 PM,Sat,Army,42,NaN,Middle Tennessee State,0,42
3,1,"Sep 5, 2020",1:00 PM,Sat,Marshall,59,NaN,Eastern Kentucky,0,59
4,1,"Sep 5, 2020",8:00 PM,Sat,Memphis,37,NaN,Arkansas State,24,61
...,...,...,...,...,...,...,...,...,...,...
169,7,"Oct 17, 2020",3:30 PM,Sat,Troy,31,NaN,Eastern Kentucky,29,60
170,7,"Oct 17, 2020",8:00 PM,Sat,(23) Virginia Tech,40,NaN,Boston College,14,54
171,7,"Oct 17, 2020",4:00 PM,Sat,Wake Forest,40,NaN,Virginia,23,63
172,7,"Oct 17, 2020",12:00 PM,Sat,West Virginia,38,NaN,Kansas,17,55


In [253]:
completed_fixtures[completed_fixtures["Away_Pts"] == 35]

,Wk,Date,Time,Day,Home,Home_Pts,Unnamed: 7,Away,Away_Pts,Total_Pts
0,1,"Sep 3, 2020",8:00 PM,Thu,Alabama-Birmingham,45,NaN,Central Arkansas,35,80
46,3,"Sep 19, 2020",6:00 PM,Sat,Southern Methodist,65,@,North Texas,35,100
53,4,"Sep 25, 2020",8:00 PM,Fri,Texas-San Antonio,37,NaN,Middle Tennessee State,35,72
62,4,"Sep 26, 2020",12:00 PM,Sat,(5) Florida,51,@,Mississippi,35,86
65,4,"Sep 26, 2020",12:00 PM,Sat,Kansas State,38,@,(3) Oklahoma,35,73
163,7,"Oct 17, 2020",5:00 PM,Sat,North Texas,52,@,Middle Tennessee State,35,87


In [254]:
actual_away = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Home
actual_home = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Away
away_pts = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Home_Pts
home_pts = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Away_Pts
completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Away"]] = actual_away
completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Home"]] = actual_home
completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Home_Pts"]] = home_pts
completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Away_Pts"]] = away_pts
completed_fixtures = completed_fixtures.drop(['Unnamed: 7'], axis=1)
completed_fixtures = completed_fixtures.assign(Spread=completed_fixtures['Away_Pts']-completed_fixtures['Home_Pts'])
completed_fixtures

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts,Total_Pts,Spread
0,1,"Sep 3, 2020",8:00 PM,Thu,Alabama-Birmingham,45.0,Central Arkansas,35.0,80,-10.0
1,1,"Sep 3, 2020",8:00 PM,Thu,Southern Mississippi,21.0,South Alabama,32.0,53,11.0
2,1,"Sep 5, 2020",1:30 PM,Sat,Army,42.0,Middle Tennessee State,0.0,42,-42.0
3,1,"Sep 5, 2020",1:00 PM,Sat,Marshall,59.0,Eastern Kentucky,0.0,59,-59.0
4,1,"Sep 5, 2020",8:00 PM,Sat,Memphis,37.0,Arkansas State,24.0,61,-13.0
...,...,...,...,...,...,...,...,...,...,...
169,7,"Oct 17, 2020",3:30 PM,Sat,Troy,31.0,Eastern Kentucky,29.0,60,-2.0
170,7,"Oct 17, 2020",8:00 PM,Sat,(23) Virginia Tech,40.0,Boston College,14.0,54,-26.0
171,7,"Oct 17, 2020",4:00 PM,Sat,Wake Forest,40.0,Virginia,23.0,63,-17.0
172,7,"Oct 17, 2020",12:00 PM,Sat,West Virginia,38.0,Kansas,17.0,55,-21.0


In [255]:
completed_fixtures.loc[:,"Home"].unique()
completed_fixtures[1:30]

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts,Total_Pts,Spread
1,1,"Sep 3, 2020",8:00 PM,Thu,Southern Mississippi,21.0,South Alabama,32.0,53,11.0
2,1,"Sep 5, 2020",1:30 PM,Sat,Army,42.0,Middle Tennessee State,0.0,42,-42.0
3,1,"Sep 5, 2020",1:00 PM,Sat,Marshall,59.0,Eastern Kentucky,0.0,59,-59.0
4,1,"Sep 5, 2020",8:00 PM,Sat,Memphis,37.0,Arkansas State,24.0,61,-13.0
5,1,"Sep 5, 2020",7:30 PM,Sat,North Texas,57.0,Houston Baptist,31.0,88,-26.0
6,1,"Sep 5, 2020",4:30 PM,Sat,Texas State,24.0,Southern Methodist,31.0,55,7.0
7,1,"Sep 5, 2020",9:00 PM,Sat,Texas-El Paso,24.0,Stephen F. Austin,14.0,38,-10.0
8,1,"Sep 7, 2020",8:00 PM,Mon,Navy,3.0,Brigham Young,55.0,58,52.0
10,2,"Sep 10, 2020",8:00 PM,Thu,Miami (FL),31.0,Alabama-Birmingham,14.0,45,-17.0
11,2,"Sep 12, 2020",12:00 PM,Sat,Appalachian State,35.0,Charlotte,20.0,55,-15.0


In [256]:
completed_fixtures["Home"] = completed_fixtures['Home'].str.split().str.join(' ').str.lstrip("1234567890() ")
completed_fixtures[1:30]

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts,Total_Pts,Spread
1,1,"Sep 3, 2020",8:00 PM,Thu,Southern Mississippi,21.0,South Alabama,32.0,53,11.0
2,1,"Sep 5, 2020",1:30 PM,Sat,Army,42.0,Middle Tennessee State,0.0,42,-42.0
3,1,"Sep 5, 2020",1:00 PM,Sat,Marshall,59.0,Eastern Kentucky,0.0,59,-59.0
4,1,"Sep 5, 2020",8:00 PM,Sat,Memphis,37.0,Arkansas State,24.0,61,-13.0
5,1,"Sep 5, 2020",7:30 PM,Sat,North Texas,57.0,Houston Baptist,31.0,88,-26.0
6,1,"Sep 5, 2020",4:30 PM,Sat,Texas State,24.0,Southern Methodist,31.0,55,7.0
7,1,"Sep 5, 2020",9:00 PM,Sat,Texas-El Paso,24.0,Stephen F. Austin,14.0,38,-10.0
8,1,"Sep 7, 2020",8:00 PM,Mon,Navy,3.0,Brigham Young,55.0,58,52.0
10,2,"Sep 10, 2020",8:00 PM,Thu,Miami (FL),31.0,Alabama-Birmingham,14.0,45,-17.0
11,2,"Sep 12, 2020",12:00 PM,Sat,Appalachian State,35.0,Charlotte,20.0,55,-15.0


## 1.1.0 Helper Functions 

Now that the elementary exploration has been finished, we can use the work above to create the elementary functions that will get us the data we need.

In [284]:
def create_url(years):
  temp = []
  for i in years:
    s = "https://www.sports-reference.com/cfb/years/{yearnumber}-schedule.html".format(yearnumber=i)
    temp.append(s)
  return temp

def get_data(url):
  raw = pd.read_html(url, header=0)
  df = raw[0]
  completed_fixtures = df.drop(df[df.Pts=='Pts'].index)
  completed_fixtures = completed_fixtures.rename(columns={'Pts':'Home_Pts', 'Pts.1':'Away_Pts','Winner':'Home','Loser':'Away'})
  completed_fixtures = completed_fixtures.drop(['Rk', 'Notes'], axis=1)
  actual_away = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Home
  actual_home = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Away
  away_pts = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Home_Pts
  home_pts = completed_fixtures[completed_fixtures.loc[:,"Unnamed: 7"]=="@"].Away_Pts
  completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Away"]] = actual_away
  completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Home"]] = actual_home
  completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Home_Pts"]] = home_pts
  completed_fixtures.loc[completed_fixtures["Unnamed: 7"]=="@", ["Away_Pts"]] = away_pts
  completed_fixtures = completed_fixtures.drop(['Unnamed: 7'], axis=1)
  completed_fixtures["Home"] = completed_fixtures['Home'].str.split().str.join(' ').str.lstrip("1234567890() ")
  completed_fixtures["Away"] = completed_fixtures['Away'].str.split().str.join(' ').str.lstrip("1234567890() ")
  return completed_fixtures

def get_completed_fixtures(full_fixtures):
  completed_fixtures = full_fixtures.dropna(subset=['Home_Pts'])
  completed_fixtures = completed_fixtures.astype({'Home_Pts':'int32'})
  completed_fixtures = completed_fixtures.astype({'Away_Pts':'int32'})
  completed_fixtures = completed_fixtures.assign(Total_Pts = completed_fixtures['Home_Pts']+completed_fixtures['Away_Pts'])
  completed_fixtures = completed_fixtures.assign(Spread=completed_fixtures['Away_Pts']-completed_fixtures['Home_Pts'])
  return completed_fixtures



In [275]:
full_fixtures = get_data("https://www.sports-reference.com/cfb/years/2019-schedule.html")
full_fixtures

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts
0,1,"Aug 24, 2019",7:00 PM,Sat,Florida,24,Miami (FL),20
1,1,"Aug 24, 2019",10:30 PM,Sat,Hawaii,45,Arizona,38
3,2,"Aug 29, 2019",8:00 PM,Thu,Alabama-Birmingham,24,Alabama State,19
4,2,"Aug 29, 2019",10:00 PM,Thu,Arizona State,30,Kent State,7
5,2,"Aug 29, 2019",7:00 PM,Thu,Bowling Green State,46,Morgan State,3
...,...,...,...,...,...,...,...,...
903,21,"Jan 2, 2020",7:00 PM,Thu,Tennessee,23,Indiana,22
904,21,"Jan 3, 2020",3:30 PM,Fri,Ohio,30,Nevada,21
905,21,"Jan 4, 2020",11:30 AM,Sat,Tulane,30,Southern Mississippi,13
906,21,"Jan 6, 2020",7:30 PM,Mon,Louisiana,27,Miami (OH),17


In [260]:
get_completed_fixtures(full_fixtures)[:]

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts,Total_Pts,Spread
0,1,"Sep 3, 2020",8:00 PM,Thu,Alabama-Birmingham,45,Central Arkansas,35,80,-10
1,1,"Sep 3, 2020",8:00 PM,Thu,Southern Mississippi,21,South Alabama,32,53,11
2,1,"Sep 5, 2020",1:30 PM,Sat,Army,42,Middle Tennessee State,0,42,-42
3,1,"Sep 5, 2020",1:00 PM,Sat,Marshall,59,Eastern Kentucky,0,59,-59
4,1,"Sep 5, 2020",8:00 PM,Sat,Memphis,37,Arkansas State,24,61,-13
...,...,...,...,...,...,...,...,...,...,...
169,7,"Oct 17, 2020",3:30 PM,Sat,Troy,31,Eastern Kentucky,29,60,-2
170,7,"Oct 17, 2020",8:00 PM,Sat,Virginia Tech,40,Boston College,14,54,-26
171,7,"Oct 17, 2020",4:00 PM,Sat,Wake Forest,40,Virginia,23,63,-17
172,7,"Oct 17, 2020",12:00 PM,Sat,West Virginia,38,Kansas,17,55,-21


In [295]:
years = list(range(2017,2021))
df2017 = get_completed_fixtures(get_data(create_url(years)[0]))
df2018 = get_completed_fixtures(get_data(create_url(years)[1]))
df2019 = get_completed_fixtures(get_data(create_url(years)[2]))
df2020 = get_completed_fixtures(get_data(create_url(years)[3]))

complete_dfs = [df2017,df2018,df2019,df2020]
complete_fixture_df = pd.concat(complete_dfs, ignore_index=True)

In [297]:
complete_fixture_df[:]

,Wk,Date,Time,Day,Home,Home_Pts,Away,Away_Pts,Total_Pts,Spread
0,1,"Aug 26, 2017",3:00 PM,Sat,Brigham Young,20,Portland State,6,26,-14
1,1,"Aug 26, 2017",2:30 PM,Sat,Colorado State,58,Oregon State,27,85,-31
2,1,"Aug 26, 2017",6:00 PM,Sat,Massachusetts,35,Hawaii,38,73,3
3,1,"Aug 26, 2017",7:30 PM,Sat,San Jose State,22,South Florida,42,64,20
4,1,"Aug 26, 2017",10:00 PM,Sat,Stanford,62,Rice,7,69,-55
...,...,...,...,...,...,...,...,...,...,...
2810,7,"Oct 17, 2020",3:30 PM,Sat,Troy,31,Eastern Kentucky,29,60,-2
2811,7,"Oct 17, 2020",8:00 PM,Sat,Virginia Tech,40,Boston College,14,54,-26
2812,7,"Oct 17, 2020",4:00 PM,Sat,Wake Forest,40,Virginia,23,63,-17
2813,7,"Oct 17, 2020",12:00 PM,Sat,West Virginia,38,Kansas,17,55,-21
